# LLM Output Configuration**
**Modèle** : *Llama-3.2-1B (Instruct)* via **Ollama**  
**Objectif** : Comprendre et maîtriser les paramètres qui contrôlent la **sortie** du modèle :
- Output length  
- Sampling controls  
- Temperature  
- Top‑K et Top‑P  
- Putting it all together


## Prérequis
- Ollama lancé en local (serveur actif) et package Python `ollama` installé.  
- Modèle : `llama3.2:1b` disponible.  

In [2]:
# (Optionnel) Installer/mettre à jour le client Python Ollama
# !pip install -q --upgrade ollama

In [1]:
import ollama

def assert_ollama_ready():
    try:
        _ = ollama.list()
        print("✅ Ollama est accessible en local.")
    except Exception as e:
        raise SystemExit(
            "❌ Impossible de contacter Ollama. Démarrez le serveur (ex: 'ollama serve').\n"
            f"Détails: {e}"
        )

assert_ollama_ready()

try:
    print("⏳ Vérification/téléchargement du modèle 'llama3.2:1b'...")
    ollama.pull('llama3.2:1b')
    print("✅ Modèle prêt.")
except Exception as e:
    print("ℹ️ Vérifiez le tag (ex: 'llama3.2:1b').")
    raise

✅ Ollama est accessible en local.
⏳ Vérification/téléchargement du modèle 'llama3.2:1b'...
✅ Modèle prêt.


## Fonctions utilitaires
Nous utilisons une fonction d’aide pour **lancer** des prompts avec différents paramètres et afficher un bref **récapitulatif**.

In [8]:
from typing import Dict, Optional
import textwrap

MODEL = 'llama3.2:1b'

def run(prompt: str, options: Optional[Dict] = None):
    options = options or {}
    r = ollama.chat(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        options=options
    )
    out = r["message"]["content"]
    return out

def show(label: str, prompt: str, options: Dict):
    print("\n=== " + label + " ===")
    print("Options:", {k: options[k] for k in sorted(options.keys())})
    out = run(prompt, options)
    print("\nSortie:\n" + textwrap.fill(out, width=100))
    print(f"\nLongueur: {len(out.split())} mots, {len(out)} caractères")

## Prompt commun pour les expériences
On utilisera des prompts courts pour visualiser l’impact des paramètres.

In [5]:
PROMPT_SHORT = (
    "Explique en 2–3 phrases ce qu'est un modèle de langage, pour un public non technique."
)
PROMPT_CREATIVE = (
    "Écris une mini-description originale (2–3 phrases) d'un café de quartier un matin de pluie."
)

## 1) Output length
Le paramètre **`num_predict`** fixe le **nombre maximal de tokens générés** (le modèle peut s’arrêter plus tôt).  
On peut aussi utiliser `stop` pour forcer un **arrêt** lorsqu’une séquence apparaît.

**À retenir** :
- `num_predict` = *plafond* de tokens générés (pas une garantie de longueur exacte).  
- `stop` = *séquence d’arrêt* (ex. `"\n\n"`, `"###"`).

In [11]:
show("num_predict=32 (court)", PROMPT_SHORT, {"num_predict": 32, "temperature": 0.2, "seed": 7})
show("num_predict=200 (long)", PROMPT_SHORT, {"num_predict": 200, "temperature": 0.2, "seed": 7})

# Exemple d'arrêt sur séquence 'FIN' (si le modèle la produit)
show("stop=['FIN']", PROMPT_CREATIVE, {"num_predict": 200, "temperature": 0.7, "stop": ["FIN"], "seed": 7})


=== num_predict=32 (court) ===
Options: {'num_predict': 32, 'seed': 7, 'temperature': 0.2}

Sortie:
Un modèle de langage est une forme d'information qui permet à des ordinateurs de comprendre et de
générer du texte ou des images en fonction

Longueur: 25 mots, 140 caractères

=== num_predict=200 (long) ===
Options: {'num_predict': 200, 'seed': 7, 'temperature': 0.2}

Sortie:
Un modèle de langage est une forme d'information qui permet de communiquer des idées ou des
informations de manière claire et précise, souvent utilisée dans les réseaux sociaux, les
applications mobiles et les sites web. Il s'agit d'un ensemble de règles et de structures pour
organiser et exprimer des concepts, ce qui en fait une forme de langage unique et spécifique.

Longueur: 61 mots, 370 caractères

=== stop=['FIN'] ===
Options: {'num_predict': 200, 'seed': 7, 'stop': ['FIN'], 'temperature': 0.7}

Sortie:
Le café du quartier, un endroit de rêve à l'abri de la pluie, était silencieux et calme, les patrons
assis

## 2) Sampling controls (panorama)
Les **contrôles d’échantillonnage** déterminent *comment* le prochain token est choisi :

- **Greedy** (pas d’échantillonnage) : `temperature=0` **et** `top_k=1` → le modèle prend toujours le meilleur token.  
- **Sampling** : on *échantillonne* selon une distribution influencée par `temperature`, `top_k`, `top_p`.  
- **Seed** : fixe la graine pseudo-aléatoire pour la **reproductibilité** partielle.

In [14]:
show("Greedy (deterministe)", PROMPT_CREATIVE, {"temperature": 0.0, "top_k": 1, "num_predict": 120, "seed": 42})
show("Sampling (temp=0.7, top_k=40, top_p=0.9)", PROMPT_CREATIVE, {"temperature": 0.7, "top_k": 40, "top_p": 0.9, "num_predict": 120, "seed": 42})


=== Greedy (deterministe) ===
Options: {'num_predict': 120, 'seed': 42, 'temperature': 0.0, 'top_k': 1}

Sortie:
Dans le quartier, les fenêtres éclairées par la lueur du matin se couvrent d'une ombre sombre,
tandis que le bruit de la pluie frappe contre les toits et les murs. Le café, un refuge pour ceux
qui cherchent à s'abriter, est rempli de gens qui discutent et rient dans le silence de la matinée
pluvieuse. Les glaçons dans les tasses tremblent légèrement sous leurs mains, tandis que les aromas
du café

Longueur: 71 mots, 399 caractères

=== Sampling (temp=0.7, top_k=40, top_p=0.9) ===
Options: {'num_predict': 120, 'seed': 42, 'temperature': 0.7, 'top_k': 40, 'top_p': 0.9}

Sortie:
Le café de quartier, un endroit à l'ombre des arcades, se réveille lentement sous la pluie
fatiguante du matin. Les fenêtres éclairées par le soleil passé font des ombres mystérieuses sur les
murs bruyants, tandis que les patrons et les clients s'étirent avec déconfort pour trouver leurs
chaises. La mu

## 3) Temperature
**`temperature`** étale ou concentre la distribution :  
- **0.0–0.2** : sobre/prévisible  
- **0.3–0.7** : équilibré  
- **0.8–1.2** : créatif/variable

On garde `seed` constant pour mieux comparer.

In [17]:
for t in [0.0, 0.3, 0.7, 1.0]:
    show(f"temperature={t}", PROMPT_CREATIVE, {"temperature": t, "top_k": 40, "top_p": 0.9, "num_predict": 120, "seed": 123})


=== temperature=0.0 ===
Options: {'num_predict': 120, 'seed': 123, 'temperature': 0.0, 'top_k': 40, 'top_p': 0.9}

Sortie:
Dans le quartier, les fenêtres éclairées par la lueur du matin se couvrent d'une ombre sombre,
tandis que le bruit de la pluie frappe contre les toits et les murs. Le café, un refuge pour ceux
qui cherchent à s'abriter, est rempli de gens qui discutent et rient dans le silence de la matinée
pluvieuse. Les glaçons dans les tasses tremblent légèrement sous leurs mains, tandis que les aromas
du café

Longueur: 71 mots, 399 caractères

=== temperature=0.3 ===
Options: {'num_predict': 120, 'seed': 123, 'temperature': 0.3, 'top_k': 40, 'top_p': 0.9}

Sortie:
Le café de quartier, un endroit secret où les gens se réunissent pour partager des conversations et
des rires malgré la pluie qui tombe à l'ouest. Les fenêtres éclairées par le soleil couchant créent
une lumière douce qui accueille les clients dans son intérieur chaud et confortable.

Longueur: 47 mots, 284 caractèr

## 4) Top‑K et Top‑P
- **Top‑K** : on restreint la **fenêtre** aux *K* meilleurs tokens.  
- **Top‑P** (*nucleus*) : on inclut les meilleurs tokens jusqu’à atteindre une **masse de probabilité** *P*.  

**Effets typiques** :
- *K petit* ou *P faible* → sorties plus **conservatrices**, parfois répétitives.  
- *K grand* ou *P haut* → plus de **diversité**, risque d’incohérence si trop élevé.

In [ ]:
show("top_k=10 (conservateur)", PROMPT_CREATIVE, {"temperature": 0.7, "top_k": 10, "top_p": 0.95, "num_predict": 120, "seed": 9})
show("top_k=1000 (large)", PROMPT_CREATIVE, {"temperature": 0.7, "top_k": 1000, "top_p": 0.95, "num_predict": 120, "seed": 9})

show("top_p=0.3 (nucleus serré)", PROMPT_CREATIVE, {"temperature": 0.7, "top_k": 100, "top_p": 0.3, "num_predict": 120, "seed": 9})
show("top_p=1.0 (quasi complet)", PROMPT_CREATIVE, {"temperature": 0.7, "top_k": 100, "top_p": 1.0, "num_predict": 120, "seed": 9})

### Température et distribution de probabilités

Le modèle prédit pour chaque token possible une **logit** (valeur brute).  
Ces logits passent ensuite par une fonction *softmax* pour donner une distribution de probabilités :


![alt text](image1.png "Title")



- **T = 1** → aucune modification.  
- **T < 1** → les différences entre probabilités sont **amplifiées** (plus déterministe).  
- **T > 1** → les différences sont **aplaties** (plus aléatoire).  

---

### Exemple

Supposons que le modèle produise les logits suivants :  
- "chat" : 5.0  
- "chien" : 4.0  
- "dragon" : 2.0  

Après softmax :

- **T = 1** → chat ≈ 70%, chien ≈ 25%, dragon ≈ 5%  
- **T = 0.5** → chat ≈ 90%, chien ≈ 9%, dragon ≈ 1%  
- **T = 2.0** → chat ≈ 50%, chien ≈ 30%, dragon ≈ 20%

## 5) Putting it all together
Des **presets** selon l’objectif :
- **Deterministic QA** : réponses stables pour tests/CI.
- **Balanced Summary** : équilibre entre concision et variation.
- **Creative Rewrite** : plus de liberté stylistique.
- **Short Answer** : contraintes fortes sur la longueur.


In [18]:
PRESETS = {
    "deterministic_qa": {"temperature": 0.0, "top_k": 1, "top_p": 1.0, "num_predict": 160, "seed": 1234},
    "balanced_summary": {"temperature": 0.3, "top_k": 40, "top_p": 0.9, "num_predict": 200, "seed": 2024},
    "creative_rewrite": {"temperature": 0.9, "top_k": 100, "top_p": 0.95, "num_predict": 220, "seed": 7},
    "short_answer": {"temperature": 0.2, "top_k": 20, "top_p": 0.8, "num_predict": 40, "seed": 77},
}

PROMPT_QA = "Réponds en 3 phrases : quelle est la différence entre IA générative et IA discriminative ?"
PROMPT_SUMMARY = (
    "Résume ce texte en 4 puces courtes : L'IA générative produit de nouveaux contenus à partir de modèles apprenant"
    " des distributions, tandis que l'IA discriminative classe ou prédit des labels. Les deux se complètent dans"
    " des pipelines modernes."
)
PROMPT_REWRITE = "Réécris cette phrase avec un ton poétique : 'Un modèle aide à rédiger des idées plus vite.'"
PROMPT_SHORT = "En une phrase, donne un conseil pour améliorer un prompt."

show("preset: deterministic_qa", PROMPT_QA, PRESETS["deterministic_qa"])
show("preset: balanced_summary", PROMPT_SUMMARY, PRESETS["balanced_summary"])
show("preset: creative_rewrite", PROMPT_REWRITE, PRESETS["creative_rewrite"])
show("preset: short_answer", PROMPT_SHORT, PRESETS["short_answer"])


=== preset: deterministic_qa ===
Options: {'num_predict': 160, 'seed': 1234, 'temperature': 0.0, 'top_k': 1, 'top_p': 1.0}

Sortie:
L'IA générative est conçue pour créer de nouvelles données ou modèles, tandis que l'IA
discriminative est conçue pour prédire des résultats basés sur des données existantes.

Longueur: 26 mots, 173 caractères

=== preset: balanced_summary ===
Options: {'num_predict': 200, 'seed': 2024, 'temperature': 0.3, 'top_k': 40, 'top_p': 0.9}

Sortie:
Voici un résumé du texte en 4 points courtes :  - L'IA générative produit de nouveaux contenus à
partir de modèles apprenant. - Cela implique la production de contenu à partir d'un modèle qui a
appris sur une distribution spécifique. - L'IA discriminative, quant à elle, classe ou prédit des
labels. - Les deux techniques se complètent dans des pipelines modernes pour générer et classifier
du contenu.

Longueur: 67 mots, 402 caractères

=== preset: creative_rewrite ===
Options: {'num_predict': 220, 'seed': 7, 'temperatur

## Bonnes pratiques
- Baissez la **température** pour des formats stricts (JSON/YAML).  
- Ajustez **`num_predict`** selon la *longueur attendue* et coupez via `stop` si besoin.  
- Évitez des valeurs extrêmes **top_k/top_p** qui dégradent la cohérence.

## À retenir
- `num_predict` : longueur maximale de génération ; `stop` pour arrêter proprement.  
- **Sampling controls** : `temperature`, `top_k`, `top_p` façonnent la diversité.  
- **Presets** : partez d’un réglage type, puis affinez pour votre cas d’usage.